In [ ]:
import datetime
from abc import ABC, abstractmethod

# UTILITÁRIOS

def mensagem(msg):
    print(f"\n{msg}")

# MODELOS

class Historico:
    def __init__(self):
        self.transacoes = []

    def adicionar_transacao(self, tipo, valor):
        self.transacoes.append({
            "tipo": tipo,
            "valor": valor,
            "data": datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        })

    def mostrar(self):
        if not self.transacoes:
            print("Nenhuma transação realizada.")
        else:
            for t in self.transacoes:
                print(f"{t['data']} - {t['tipo']}: R$ {t['valor']:.2f}")

class Conta:
    def __init__(self, agencia, numero, cliente):
        self.agencia = agencia
        self.numero = numero
        self.cliente = cliente
        self.saldo = 0.0
        self.historico = Historico()

    def depositar(self, valor):
        if valor > 0:
            self.saldo += valor
            self.historico.adicionar_transacao("Depósito", valor)
            mensagem(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Valor inválido para depósito.")
        return False

    def sacar(self, valor):
        if 0 < valor <= self.saldo:
            self.saldo -= valor
            self.historico.adicionar_transacao("Saque", valor)
            mensagem(f"Saque de R$ {valor:.2f} realizado com sucesso.")
            return True
        mensagem("Saque não autorizado. Verifique saldo e limite.")
        return False

    def extrato(self):
        print(f"\nExtrato da Conta {self.numero}:")
        self.historico.mostrar()
        print(f"Saldo atual: R$ {self.saldo:.2f}\n")

class ContaCorrente(Conta):
    def __init__(self, agencia, numero, cliente, limite_saque=500, limite_saques_dia=3):
        super().__init__(agencia, numero, cliente)
        self.limite_saque = limite_saque
        self.limite_saques_dia = limite_saques_dia
        self.saques_realizados = 0
        self.data_ultimo_saque = None

    def sacar(self, valor):
        hoje = datetime.date.today()
        if self.data_ultimo_saque != hoje:
            self.saques_realizados = 0
            self.data_ultimo_saque = hoje

        if self.saques_realizados >= self.limite_saques_dia:
            mensagem("Limite diário de saques atingido.")
            return False

        if valor > self.limite_saque:
            mensagem(f"Valor excede o limite de saque: R$ {self.limite_saque:.2f}.")
            return False

        if super().sacar(valor):
            self.saques_realizados += 1
            return True
        return False

class Cliente:
    def __init__(self, nome, cpf, nascimento, endereco):
        self.nome = nome
        self.cpf = cpf
        self.nascimento = nascimento
        self.endereco = endereco
        self.contas = []

    def adicionar_conta(self, conta):
        self.contas.append(conta)

    def recuperar_conta(self):
        return self.contas[0] if self.contas else None


class Banco:
    def __init__(self):
        self.clientes = []
        self.contas = []
        self.numero_conta = 1

    def buscar_cliente(self, cpf):
        return next((c for c in self.clientes if c.cpf == cpf), None)

    def criar_cliente(self, nome, cpf, nascimento, endereco):
        if self.buscar_cliente(cpf):
            mensagem("CPF já cadastrado.")
            return
        cliente = Cliente(nome, cpf, nascimento, endereco)
        self.clientes.append(cliente)
        mensagem("Cliente cadastrado com sucesso.")

    def criar_conta_corrente(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = ContaCorrente("0001", str(self.numero_conta), cliente)
        cliente.adicionar_conta(conta)
        self.contas.append(conta)
        self.numero_conta += 1
        mensagem("Conta criada com sucesso.")

    def depositar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.depositar(valor)

    def sacar(self, cpf, valor):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.sacar(valor)

    def ver_extrato(self, cpf):
        cliente = self.buscar_cliente(cpf)
        if not cliente:
            mensagem("Cliente não encontrado.")
            return
        conta = cliente.recuperar_conta()
        if not conta:
            mensagem("Conta não encontrada.")
            return
        conta.extrato()

    def listar_contas(self):
        if not self.contas:
            print("Nenhuma conta cadastrada.")
        for conta in self.contas:
            print(f"Agência: {conta.agencia} | Conta: {conta.numero} | Titular: {conta.cliente.nome}")

# MOCK

def testes():
    banco = Banco()
    banco.criar_cliente("João Silva", "12345678900", "01/01/1990", "Rua A, 100")
    banco.criar_conta_corrente("12345678900")
    banco.depositar("12345678900", 1000.0)
    banco.sacar("12345678900", 200.0)
    banco.ver_extrato("12345678900")
    banco.listar_contas()

if __name__ == "__main__":
    testes()



Cliente cadastrado com sucesso.

Conta criada com sucesso.

Depósito de R$ 1000.00 realizado com sucesso.

Saque de R$ 200.00 realizado com sucesso.

Extrato da Conta 1:
01/07/2025 03:20:48 - Depósito: R$ 1000.00
01/07/2025 03:20:48 - Saque: R$ 200.00
Saldo atual: R$ 800.00

Agência: 0001 | Conta: 1 | Titular: João Silva
